# 호현아 부탁한다 이거 굉장히 오래걸릴거야
- 1,000개당 1분 20초 정도 걸림
- 그걸 254만개에 수행해야 함
- 부탁함

In [ ]:
# https://frhyme.github.io/python-lib/pipeline_in_python/

import re
from chatspace import ChatSpace # Requirements;
                                # https://lsjsj92.tistory.com/494
                                # conda install pytorch torchvision cudatoolkit=9.0 -c python
                                # pip install chatspace
from konlpy.tag import Okt
import functools
spacer = ChatSpace()
okt = Okt()

stopwords = list('의가이은을로들는좀잘걍과도를자에그와한것다') + ['으로', '하다', '에서', '이다']

def composeFuncs(func1, func2):
    return lambda x : func2(func1(x))

pipes = [lambda x : re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', x),
         lambda x : spallcheck(x).checked,
         lambda x : x.replace(' ', ''),
         lambda x : spacer.space(x),
         lambda x : okt.morphs(x, stem=True, norm=True),
#          lambda x : mecab.morphs(''.join(x)),
         lambda x : [s for s in x if s not in stopwords]]

preprocessText = functools.reduce(composeFuncs, pipes)

In [ ]:
reviews5over = pd.read_csv('reviews5over.csv')

In [ ]:
reviews5over.head()

In [ ]:
reviews5over.size

In [ ]:
import pickle
import time
res = []
start, semi_start = time.time(), time.time()
for ix, text in enumerate(reviews5over['reviews'].values):
    # 1,000개마다 결과 출력
    if ix % 1000 == 0:
        if time.time() - start >= 60:
            a = (time.time() - start) / 3600
            b = 'hours'
        else:
            a = (time.time() - start) / 60
            b = 'mins'
        print(ix, '\t\tcollapse {:.2f} mins\t\tcumulative time is {:.2f} {:s}'.format(
            (time.time() - semi_start) / 60, a, b)
        semi_start = time.time()
    res.append(preprocessText(text))
    # 100,000개마다 결과 저장
    if ix % 100000 == 0:
        with open('preprocessReview_{}'.format(ix), 'wb') as f:
            pickle.dump(res, f, protocol=pickle.HIGHEST_PROTOCOL)
        res = []